### Introduction to TorchPhysics

This notebook contains a template for the joint exercise regarding physics informed deep learning and TorchPhysics.

We start by solving the Laplace equation:

\begin{align*}
    \Delta u &= 1 &&\text{ in } \Omega \\
    u &= 0 , &&\text{ on } \partial \Omega
\end{align*}

with $\Omega = (0, 1) \times (0, 1)$.

In [ ]:
# This block is for GPU selection. Please execute.
import pathlib
import os
user = int(str(pathlib.Path().resolve())[22:24])
if user <= 21: 
    gpu_device = str(user % 7) # moriarty
else: gpu_device = str(user % 4) # neptuno
os.environ["CUDA_VISIBLE_DEVICES"]= gpu_device

Next we implement the *Spaces* that appear in the problem:

In [3]:
import torch
import torchphysics as tp

X = tp.spaces.R2("")
U = tp.spaces.R1("")

Now we define our domain, the unit square:

In [4]:
omega = tp.domains.Parallelogram(X, )

For the training we need to create some points, this is handle by the *Sampler*:

In [5]:
inner_sampler = tp.samplers.RandomUniformSampler(omega, n_points=)

boundary_sampler = tp.samplers.GridSampler(omega.boundary, n_points=)

In [ ]:
fig = tp.utils.scatter(X, inner_sampler, boundary_sampler)

We need a neural network that should learn our solution:

In [7]:
model = tp.models.FCN(input_space=, output_space=, hidden=(20,20,20))

Now, we have to transform out mathematical conditions given by our PDE into corresponding training conditions. First for the differential equation itself:

In [8]:
def pde_residual(u, x):
    return tp.utils.laplacian() - 1.0

pde_cond = tp.conditions.PINNCondition()

Next for the boundary condition:

In [9]:
def boundary_residual(u):
    return 

boundary_cond = tp.conditions.PINNCondition()

Before the training we collect all conditions and choose our training procedure:

In [10]:
optim = tp.OptimizerSetting(torch.optim.Adam, lr=)
solver = tp.solver.Solver([.,.], optimizer_setting=optim)

Start the training:

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(devices=1, accelerator="gpu", # use one GPU
                     max_steps=5000, # iteration number
                     benchmark=True, # faster if input batch has constant size
                     logger=False, # for writting into tensorboard
                     enable_checkpointing=False) # saving checkpoints
trainer.fit(solver)

Lastly, we can plot the solution:

In [ ]:
plot_sampler = tp.samplers.PlotSampler(plot_domain=omega, n_points=2000)
fig = tp.utils.plot(model, lambda u : u, plot_sampler)

And compare PINNs with the FEM-solution:

In [ ]:
import numpy as np

# Read data:
coords = torch.tensor(np.load("/localdata/tomfre/SolutionData/LaplaceData/coordinates.npy"), dtype=torch.float32)
fem_sol = torch.tensor(np.load("/localdata/tomfre/SolutionData/LaplaceData/solution.npy"), dtype=torch.float32).reshape(-1, 1)

# Evaluate Network:
model_out = model(tp.spaces.Points(coords, X)).as_tensor

print("Difference to FEM in Sup-norm:")
difference_sup = torch.max(torch.abs(fem_sol - model_out))
print("Absolute:", difference_sup)
print("Relative:", difference_sup / torch.max(torch.abs(fem_sol)))